In [1]:
import sys
import os

# Get the absolute path to the repo root (parent of both src and notebook folders)
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add the repo root to the Python path
sys.path.append(repo_root)

# Now you can import from src
from src.master.data import load_and_crop_data, get_positive_negative_masks, process_segmentation_masks
from src.master.visualize import show_masks_sam, visualize_masks
from src.master.evaluate import evaluate_binary_masks

In [ ]:
img_path = "C:/Users/gtoft/OneDrive/DTU/4_Semester_AS/Master_Thesis/data/sam_inference/processed_data/meatballs/images/frame_000000.PNG"
gt_path = "C:/Users/gtoft/OneDrive/DTU/4_Semester_AS/Master_Thesis/data/sam_inference/processed_data/meatballs/gt_masks/frame_000000.txt"
sam_path = "C:/Users/gtoft/OneDrive/DTU/4_Semester_AS/Master_Thesis/data/sam_inference/processed_data/meatballs/sam_masks/frame_000000.txt"

img, masks_gt = load_and_crop_data(img_path, gt_path)

_, masks_sam = load_and_crop_data(img_path, sam_path)

visualize_masks(img, masks_gt)
visualize_masks(img, masks_sam)